<a href="https://colab.research.google.com/github/luisosmx/proyect_e2e/blob/main/pof_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install unidecode
import requests
import pandas as pd
import datetime
import unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 14.4 MB/s eta 0:00:00


In [30]:
URL = 'https://datos.cdmx.gob.mx/dataset/f2046fd5-51b5-4876-b008-bd65d95f9a02/resource/0e8ffe58-28bb-4dde-afcd-e5f5b4de4ccb/download/afluenciastc_simple_01_2023.csv'


In [31]:
  response = requests.get(URL)
  file_path = "afluenciastc_simple_01_2023.csv"
  open(file_path, "wb").write(response.content)

45944517

In [32]:
def data_extraction(url: str) -> pd.DataFrame:

  # Descarga de archivo
  response = requests.get(url)
  file_path = "afluenciastc_simple_01_2023.csv"
  open(file_path, "wb").write(response.content)

  # Leer datos del archivo csv en un DataFrame de Pandas
  df = pd.read_csv("afluenciastc_simple_01_2023.csv", encoding='ISO-8859-1')
  return df 

In [33]:
def data_tranformation(df: pd.DataFrame) -> pd.DataFrame:

  # Renombrar columnas del DataFrame
  df = df.rename(columns={
      'fecha': 'date',
      'anio': 'year',
      'mes': 'month',
      'linea': 'line',
      'estacion': 'station',
      'afluencia': 'influx'
  })

  # Convertir nombres de líneas a minúsculas
  df['line'] = df['line'].str.lower()

  # Convertir nombres de estaciones a caracteres sin acentos
  df['station'] = df['station'].apply(lambda x: unidecode.unidecode(x))

  # Agregar fecha y hora de ingestión como una nueva columna al DataFrame
  now = datetime.datetime.now()
  ingestion_date = now.strftime('%Y-%m-%d %H:%M:%S')
  df['ingestion_date'] = ingestion_date

  # Convertir múltiples columnas a cadenas de caracteres
  df = df.applymap(str)
  df['year'] = df['year'].astype('int')
  df['influx'] = df['influx'].astype('int')
  return df

In [ ]:
raw_data = data_extraction(URL)
transformed_data = data_tranformation(raw_data)
display(transformed_data)
